In [9]:
import pandas as pd
import numpy as np

In [10]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [11]:
train = train_df
test = test_df

In [12]:
train = train[train['time']<2*1e6]
train = train[train['time']!=1804691]
train = train[train['Store_Ratio']>0]
# train.shape
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [13]:
test = test[test['Store_Ratio']>0]
train = train[train['Store_Presence']>0]
test = test[test['Store_Presence']>0]

In [6]:
# train['avg_score'] = (train['Score_1']+train['Score_2']+train['Score_3']+train['Score_4'])/4 
# train['score_by_presence'] = train['avg_score']/train['Store_Presence']
# train['score_by_basket'] = train['avg_score']/train['Basket_Ratio']

In [7]:
# test['avg_score'] = (test['Score_1']+test['Score_2']+test['Score_3']+test['Score_4'])/4 
# test['score_by_presence'] = test['avg_score']/test['Store_Presence']
# test['score_by_basket'] = test['avg_score']/test['Basket_Ratio']

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss
import warnings 
warnings.filterwarnings('ignore')

In [17]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(loss_function='MultiClass', 
                             eval_metric='TotalF1', 
                             depth=7,
                             verbose=0,
                             n_estimators=800
)
  model.fit(*train_set)
  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.41224470292082255
0.41047578709442295
0.4195700008492181
0.4234317764951753
0.39696320320457806
0.3813218742128774
0.3930185184657678
0.4052356101952152
0.38569538049938346
0.3961578478234381
Mean OOF Score : 0.4024114701760898


In [38]:
temp = pd.DataFrame(final_preds)
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]
sub = sub.fillna(0)

,0,1,2,3
0,0.022960,0.722087,0.196269,0.058684
1,0.003880,0.023486,0.970499,0.002135
2,0.000536,0.038598,0.953302,0.007565
3,0.001646,0.020751,0.969149,0.008454
4,0.000135,0.002288,0.995976,0.001602


In [32]:
train_df.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity,ALL
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4,Store_Ratio0.407Basket_Ratio0.0038Category_12S...
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4,Store_Ratio0.234Basket_Ratio0.105Category_10St...
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4,Store_Ratio0.6679999999999999Basket_Ratio0.726...
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5,Store_Ratio0.184Basket_Ratio0.00561Category_14...
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3,Store_Ratio0.231Basket_Ratio0.131Category_16St...


In [ ]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''

for i in test_df.columns:
    temp1=temp1+i+train_df[i].astype(str)
train_df['ALL']=temp1
temp2=''
for i in test_df.columns:
    temp2=temp2+i+test_df[i].astype(str)
test_df['ALL']=temp2
encoding=train_df.groupby('ALL')['popularity'].median()
y_from_train=train_df['ALL'].map(encoding)
testing = pd.DataFrame()
testing['pop'] = y_from_train
testing['pop'] = testing['pop'].fillna(-999)
x = testing[testing['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [43]:
len(test_df)-(y_from_train.isna().sum())

12140

In [41]:
sub.head()

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0


In [35]:
dummy = test_df[test_df['Store_Ratio']==0]
dummy.index

Int64Index([358, 2873, 5982, 5984, 6306, 7565, 7761, 8615, 8829, 11152, 12010], dtype='int64')

In [36]:
for i in dummy.index.values:
    sub.iloc[i] =[1.0,0.0,0.0,0.0,0.0]
sub.head()

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0


In [37]:
np.sum(sub['0'])

27.0

NameError: name 'train' is not defined